# Stock Trades by Members of the US House of Representatives

This project uses public data about the stock trades made by members of the US House of Representatives. This data is collected and maintained by Timothy Carambat as part of the [House Stock Watcher](https://housestockwatcher.com/) project. The project describes itself as follows:

> With recent and ongoing investigations of incumbent congressional members being investigated for potentially violating the STOCK act. This website compiles this publicly available information in a format that is easier to digest then the original PDF source.
>
> Members of Congress must report periodic reports of their asset transactions. This website is purely for an informative purpose and aid in transparency.
>
> This site does not manipluate or censor any of the information from the original source. All data is transcribed by our community of contributors, which you can join for free by going to our transcription tool. Our moderation team takes great care in ensuring the accuracy of the information.
>
> This site is built and maintained by Timothy Carambat and supported with our contributors.

In [399]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

In [400]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import requests
import bs4
\
import lxml

Some interesting questions to consider for this data set include:

- Is there a difference in stock trading behavior between political parties? For example:
    - does one party trade more often?
    - does one party make larger trades?
    - do the two parties invest in different stocks or sectors? For instance, do Democrats invest in Tesla more than Republicans?
- What congresspeople have made the most trades?
- What companies are most traded by congresspeople?
- Is there evidence of insider trading? For example, Boeing stock dropped sharply in February 2020. Were there a suspiciously-high number of sales of Boeing before the drop?
- When are stocks bought and sold? Is there a day of the week that is most common? Or a month of the year?

### Getting the Data

The full data set of stock trade disclosures is available as a CSV or as JSON at https://housestockwatcher.com/api.

This data set does not, however, contain the political affiliation of the congresspeople. If you wish to investigate a question that relies on having this information, you'll need to find another dataset that contains it and perform a merge. *Hint*: Kaggle is a useful source of data sets.

In [401]:
r = requests.get('https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json')
df = pd.DataFrame(r.json())

In [402]:
sort_date = df.sort_values(by='disclosure_year', ascending = False)
sort_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15710 entries, 10717 to 15709
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   disclosure_year         15710 non-null  int64 
 1   disclosure_date         15710 non-null  object
 2   transaction_date        15710 non-null  object
 3   owner                   9667 non-null   object
 4   ticker                  15710 non-null  object
 5   asset_description       15706 non-null  object
 6   type                    15710 non-null  object
 7   amount                  15710 non-null  object
 8   representative          15710 non-null  object
 9   district                15710 non-null  object
 10  ptr_link                15710 non-null  object
 11  cap_gains_over_200_usd  15710 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 1.5+ MB


In [403]:
sort_date

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
10717,2022,10/11/2022,2022-09-12,self,TSLA,Tesla Inc,sale_partial,"$1,001 - $15,000",Hon. Kathy Manning,NC06,https://disclosures-clerk.house.gov/public_dis...,False
7530,2022,06/19/2022,2022-05-18,self,--,Orange CTY CA REV UTX,purchase,"$100,001 - $250,000",Hon. Scott H. Peters,CA52,https://disclosures-clerk.house.gov/public_dis...,False
7576,2022,09/02/2022,2021-03-04,self,WM,Waste Management Inc,purchase,"$1,001 - $15,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
7575,2022,09/02/2022,2021-03-04,self,VTRA,Viatris Inc,sale_full,"$1,001 - $15,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
7574,2022,09/02/2022,2021-07-28,self,VIAC,ViacomCBS Inc - Class B,purchase,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7377,2020,08/07/2020,2020-05-28,self,CRM,Salesforce.com Inc,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False
7378,2020,08/07/2020,2020-06-29,self,SFST,"Southern First Bancshares, Inc.",sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False
7379,2020,08/07/2020,2020-05-28,self,SBUX,Starbucks Corporation,sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,True
7380,2020,08/07/2020,2020-05-28,self,WMT,Walmart Inc.,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False


In [404]:
sort_date['owner'].unique()

array(['self', None, 'joint', 'dependent', '--'], dtype=object)

In [405]:
affiliation = pd.read_csv("house_members_116.csv")
affiliate = affiliation[['name', 'current_party']]
affiliate

,name,current_party
0,ralph-abraham,Republican
1,alma-adams,Democratic
2,robert-aderholt,Republican
3,pete-aguilar,Democratic
4,rick-allen,Republican
...,...,...
438,ron-wright,Republican
439,john-yarmuth,Democratic
440,ted-yoho,Republican
441,don-young,Republican


In [406]:
affiliate['name'] = affiliate['name'].str.replace('-', ' ')

C:\Users\ChristineXu\AppData\Local\Temp\ipykernel_4524\2999190261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affiliate['name'] = affiliate['name'].str.replace('-', ' ')


True

In [407]:
def clean_name(name):
    name = name.replace('Hon. ', '')
    name = name.replace('None ', '')
    name = name.replace('Mr. ', '')
    name = name.replace('Mrs. ', '')
    name = name.replace('W. ', '')
    no_middle = name.split(' ')[0] +' ' + name.split(' ')[-1]
    return no_middle.lower()
affiliate['name'] = affiliate['name'].transform(clean_name)
affiliate

C:\Users\ChristineXu\AppData\Local\Temp\ipykernel_4524\6193341.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affiliate['name'] = affiliate['name'].transform(clean_name)


,name,current_party
0,ralph abraham,Republican
1,alma adams,Democratic
2,robert aderholt,Republican
3,pete aguilar,Democratic
4,rick allen,Republican
...,...,...
438,ron wright,Republican
439,john yarmuth,Democratic
440,ted yoho,Republican
441,don young,Republican


In [408]:
'nancy pelosi' in affiliate['name'].values

True

In [409]:
'nancy pelosi'.replace('Hon. ', '')

'nancy pelosi'

In [410]:
'daniel crenshaw' in affiliate['name'].values

False

In [411]:
replace_values = {
           'rick allen': 'richard allen',
           'cynthia axne': 'cindy axne',
           'dan crenshaw': 'daniel crenshaw',
           'gregory murphy': 'greg murphy',
            'w steube': 'greg steube',
           'jim costa': 'james costa',
           'ro khanna': 'rohit khanna',
           'mike gallagher': 'michael gallagher',
           'k conaway': 'k. conaway',
           'a mceachin': 'aston mceachin',
           'ken buck': 'kenneth buck',
           'jim banks': 'james banks',
           'j hill':'james hill',
           'raul grijalva': 'raúl grijalva',
           'mario balart': 'mario diaz-balart',
           'jim hagedorn':'james hagedorn',
           'raja krishnamoorthi': 's. krishnamoorthi',
           'wm clay': 'wm. clay',
           'tom halleran': "tom o'halleran",
          }

In [412]:
sort_date['cleaned_name'] = sort_date['representative'].transform(clean_name)

In [413]:
affiliate['name'] = affiliate['name'].replace(replace_values)
'w steube' in affiliate['name'].values

C:\Users\ChristineXu\AppData\Local\Temp\ipykernel_4524\4036000746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affiliate['name'] = affiliate['name'].replace(replace_values)


False

In [414]:
with_party = sort_date.merge(affiliate, left_on = 'cleaned_name', right_on = 'name', how='left')


In [415]:
with_party[with_party['representative'].str.contains('V. Taylor')]

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,cleaned_name,name,current_party
887,2022,01/14/2022,2021-12-17,None,RNGR,Ranger Energy Services Inc Class A,exchange,"$15,001 - $50,000",Hon. Nicholas V. Taylor,TX03,https://disclosures-clerk.house.gov/public_dis...,False,nicholas taylor,NaN,NaN


In [416]:
nans = with_party[with_party['name'].isna()]
uni = nans['cleaned_name'].unique()
uni

array(['kathy manning', 'scott franklin', 'maria salazar',
       'christopher jacobs', 'marjorie greene', 'bill pascrell',
       'august pfluger', 'patrick fallon', 'pete sessions',
       'victoria spartz', 'andrew garbarino', 'nicholas taylor',
       'david cawthorn', 'jake auchincloss', 'deborah ross',
       'marie newman', 'blake moore', 'michael garcia',
       'diana harshbarger', 'sara jacobs', 'peter meijer',
       'ashley arenholz', 'felix moore', 'stephanie bice', 'neal facs'],
      dtype=object)

In [417]:
'tj cox' in affiliate['name'].values

True

In [418]:
d = 'Democratic'
r = 'Republican'
mannul_fill_party = {
                    'kathy manning': d, 'scott franklin': r,
                    'maria salazar': r, 'christopher jacobs': r,
                    'marjorie greene': r, 'bill pascrell': d,
                     'august pfluger': r, 'patrick fallon': r,
                     'pete sessions': r, 'victoria spartz': r,
                     'andrew garbarino': r,'david cawthorn': r,
                     'jake auchincloss': d, 'deborah ross': d,
                     'marie newman': d, 'blake moore': r,
                     'michael garcia': r, 'diana harshbarger': r,
                     'sara jacobs': d, 'peter meijer': r,
                     'ashley arenholz':r,'felix moore': r,
                     'stephanie bice': r, 'neal facs': r
                    }
nicholas_taylors = {"Hon. Nicholas Van Taylor": r,
                   "Hon. Nicholas V. Taylor": d}

In [419]:
def mannul_fill(row):
    if row['cleaned_name'] in mannul_fill_party.keys():
        return mannul_fill_party[row['cleaned_name']]
    elif row['representative'] in nicholas_taylors.keys():
        return nicholas_taylors[row['representative']]
    else:
        return row['current_party']

In [423]:
full_party = with_party.apply(mannul_fill, axis=1)
full_party

0        Democratic
1        Democratic
2        Republican
3        Republican
4        Republican
            ...    
15705    Republican
15706    Republican
15707    Republican
15708    Republican
15709    Republican
Length: 15710, dtype: object

In [424]:
with_party['current_party'] = full_party

In [428]:
with_party['current_party'].isna().sum()

0


### Cleaning and EDA

- Clean the data.
    - Certain fields have "missing" data that isn't labeled as missing. For example, there are fields with the value "--." Do some exploration to find those values and convert them to null values.
    - You may also want to clean up the date columns to enable time-series exploration.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.


### Assessment of Missingness

- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [ ]:
# TODO

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Test / Permutation Test

In [ ]:
# TODO